Constants from this site: https://www.timeanddate.com/sun/usa/washington-dc?month=3&year=2024

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time

# Constants
max_sun_hours = 14
max_sun_minutes = 54
max_sunrise = '04:42'
max_sunset = '19:36'
max_date = '06-21'

min_sun_hours = 9
min_sun_minutes = 26
min_sunrise = '07:22'
min_sunset = '16:48'
min_date = '12-21'

# Helper function to convert time string to minutes since midnight
def time_to_minutes(t):
    h, m = t.split(':')
    return int(h) * 60 + int(m)

# Interpolation function
def interpolate_time(day_of_year, max_value, min_value, max_day_of_year, min_day_of_year):
    # Linear interpolation
    return min_value + (max_value - min_value) * (np.cos((day_of_year - max_day_of_year) * 2 * np.pi / 365) + 1) / 2

# Prepare data range for the specified years
start_date = datetime(2016, 1, 1)
end_date = datetime(2024, 1, 1, 23, 59)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Create DataFrame
df = pd.DataFrame(date_range, columns=['datetime'])
df['day_of_year'] = df['datetime'].dt.dayofyear
df['year'] = df['datetime'].dt.year

# Constants for calculations
max_sunrise_minutes = time_to_minutes(max_sunrise)
max_sunset_minutes = time_to_minutes(max_sunset)
min_sunrise_minutes = time_to_minutes(min_sunrise)
min_sunset_minutes = time_to_minutes(min_sunset)

# Update day_of_year calculations to be dynamic
def get_day_of_year(date_str, year):
    return datetime.strptime(f"{date_str}-{year}", '%m-%d-%Y').timetuple().tm_yday

# Apply dynamic day_of_year for max and min calculations
df['max_day_of_year'] = df['year'].apply(lambda x: get_day_of_year(max_date, x))
df['min_day_of_year'] = df['year'].apply(lambda x: get_day_of_year(min_date, x))

# Calculate sunrise and sunset times for each day
df['sunrise'] = df.apply(lambda row: interpolate_time(row['day_of_year'], max_sunrise_minutes, min_sunrise_minutes, row['max_day_of_year'], row['min_day_of_year']), axis=1)
df['sunset'] = df.apply(lambda row: interpolate_time(row['day_of_year'], max_sunset_minutes, min_sunset_minutes, row['max_day_of_year'], row['min_day_of_year']), axis=1)

# Determine if the sun is up
df['is_sun_up'] = df.apply(lambda row: time_to_minutes(row['datetime'].strftime('%H:%M')) >= row['sunrise'] and time_to_minutes(row['datetime'].strftime('%H:%M')) <= row['sunset'], axis=1)

# Show sample data
df[['datetime', 'is_sun_up']].head() # Display first few rows to check
# Optionally, save to CSV
df[['datetime', 'is_sun_up']].to_csv('sun_data_2016_2024.csv', index=False)


C:\Users\15714\AppData\Local\Temp\ipykernel_17900\54838257.py:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='H')
